In [1]:
!wget https://huggingface.co/danjacobellis/dance/resolve/main/lsdir_ft_rgb_f16c12.pth

--2025-04-14 14:36:09--  https://huggingface.co/danjacobellis/dance/resolve/main/lsdir_ft_rgb_f16c12.pth
Resolving huggingface.co (huggingface.co)... 99.86.102.115, 99.86.102.9, 99.86.102.128, ...
Connecting to huggingface.co (huggingface.co)|99.86.102.115|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/d8/1e/d81e83e7bf11bcd7e71f3a8e05ae4eaab4f578a5377b018a767bd640ffcc0f0f/60287be8ff53f064f2f65afcb3fb5e0d382bfeecbaf2f4c4a14b29eed892fb4a?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27lsdir_ft_rgb_f16c12.pth%3B+filename%3D%22lsdir_ft_rgb_f16c12.pth%22%3B&Expires=1744641369&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDY0MTM2OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Q4LzFlL2Q4MWU4M2U3YmYxMWJjZDdlNzFmM2E4ZTA1YWU0ZWFhYjRmNTc4YTUzNzdiMDE4YTc2N2JkNjQwZmZjYzBmMGYvNjAyODdiZThmZjUzZjA2NGYyZjY1YWZjYjNmYjVlMGQzODJiZmVlY2JhZjJmNGM0YTE0YjI5ZWVkODkyZmI0

In [2]:
import torch
import torch.nn as nn
import PIL.Image
import io
import numpy as np
import datasets
import time
from types import SimpleNamespace
from datasets import Dataset
from torchvision.transforms.v2 import CenterCrop
from torchvision.transforms.v2.functional import to_pil_image, pil_to_tensor
from autocodec.codec import AutoCodecND, latent_to_pil, pil_to_latent

In [3]:
def compress_and_evaluate(sample, quality=0.1):
    img = sample['image']
    x_orig = pil_to_tensor(img).to(device).unsqueeze(0).to(torch.float) / 127.5 - 1.0
    orig_dim = x_orig.numel() 

    t0 = time.time()
    with torch.no_grad():
        z = model.encode(x_orig)
        latent = model.quantize.compand(z).round()
    webp = latent_to_pil(latent.cpu(), n_bits=8, C=3)
    buff = io.BytesIO()
    webp[0].save(buff, format='WEBP', lossless=True)
    neural_encode_time = time.time() - t0
    
    return {
        'neural_encode_time': neural_encode_time,
        'mp':orig_dim/3e6
    }

In [4]:
device = "cpu"
valid_dataset = datasets.load_dataset("danjacobellis/LSDIR_val", split='validation').select(range(10))

In [10]:
checkpoint = torch.load('lsdir_ft_rgb_f16c12.pth', map_location=device,weights_only=False)
config = checkpoint['config']
state_dict = checkpoint['state_dict']

model = AutoCodecND(
    dim=2,
    input_channels=config.input_channels,
    J=int(np.log2(config.F)),
    latent_dim=config.latent_dim,
    encoder_depth = config.encoder_depth,
    encoder_kernel_size=config.encoder_kernel_size,
    decoder_depth = config.decoder_depth,
    lightweight_encode=config.lightweight_encode,
    lightweight_decode=config.lightweight_decode,
).to(device)
model.load_state_dict(state_dict)
del model.decoder_blocks
model.eval();
print(sum(p.numel() for p in model.parameters())/1e6)

0.499356


In [11]:
model = torch.compile(model)

In [13]:
r = valid_dataset.map(lambda batch: compress_and_evaluate(batch,0.45)).with_format('torch')
r['mp'] / r['neural_encode_time']

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

tensor([0.3628, 0.5137, 0.4877, 0.4579, 0.4626, 0.4794, 0.4651, 0.5010, 0.4909,
        0.5163])

In [15]:
(r['mp'] / r['neural_encode_time']).mean()

tensor(0.4738)